In [1]:
!pip install llmlingua accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
!pip install langchain boto3 rouge-score

In [126]:
import boto3
bedrock = boto3.client('bedrock-runtime',
                      region_name='us-east-1')

In [ ]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor()

In [4]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader('https://openai.com/research/overview')

In [5]:
docs = loader.load()

In [6]:
docs[0]

Document(page_content="\n\n\nResearch\n\n\n\n\n\n\n\n\n\n\n\n\n\nCloseSearch Submit Skip to main contentSite NavigationResearchOverviewIndexGPT-4DALL·E 3APIOverviewData privacyPricingDocsChatGPTOverviewEnterpriseTry ChatGPTSafetyCompanyAboutBlogCareersResidencyCharterSecurityCustomer storiesSearch Navigation quick links Log inTry ChatGPTMenu Mobile Navigation CloseSite NavigationResearchOverviewIndexGPT-4DALL·E 3APIOverviewData privacyPricingDocsChatGPTOverviewEnterpriseTry ChatGPTSafetyCompanyAboutBlogCareersResidencyCharterSecurityCustomer stories Quick Links Log inTry ChatGPTSearch Submit Pioneering research on the path to AGIWe believe our research will eventually lead to artificial general intelligence, a system that can solve human-level problems. Building safe and beneficial AGI is our mission.Quick linksView research indexLearn about safetySafely aligning powerful AI systems is one of the most important unsolved problems for our mission. Techniques like learning from human feed

In [109]:
instruction = "You are a helpful article research assistant"
question = "Using two sentences, tell me what OpenAI is currently researching based on the article, do not use a preamble"
context = docs[0].page_content

In [116]:
prompt = instruction + '\n\n' + context + '\n\n' + question
len(prompt.split())

544

In [117]:
compressed_prompt = llm_lingua.compress_prompt(
    context = [context],
    instruction = instruction,
    question = question,
    target_token = 350,
    rank_method = 'longllmlingua'
)

In [118]:
len(compressed_prompt['compressed_prompt'].split())

151

In [119]:
compressed_prompt

{'compressed_prompt': 'You are a helpful article research assistant\n\n\n\nearch\n\nCloseSearchmit contentSite NavigationearchviewIndexG-4DALLE 3APIviewDataacyingocsPTviewprisePTetyAboutersRescyCharterSecurity storiesSearch Navigation links inPTMenu Mobile Navigation Close storiesPTSearchmit Pering research theIWe believe will to, system that can solve humanlevel. Building andicial. linksView research index about safetyafelying systems important problems our. Te like are closer are us.Joshiamer OpenAIF areasWe build our a called which train.Textur are advanced language processing generate, text high of.ing to instructionsve are following user intent than.izing withWe a toize entire books with human.Language areersWe,.Imageuring to which makes a images that creating.Hical-P that explicitly representations imageity with in and.We networkptions concepts.:’P whichns language.Audio on to and original.cingper neural that level on.Jre music as and.M generateastur a. researchImagePT00ving a99g

In [120]:
from langchain.llms import Bedrock
llm = Bedrock(client=bedrock, model_id='anthropic.claude-v2')

In [123]:
from pprint import pprint
original = llm(prompt)
condensed = llm(compressed_prompt['compressed_prompt'])
pprint(f"Original: {original}")
print('#' * 60)
pprint(f"Condensed: {condensed}")

('Original:  Based on the article, OpenAI is currently researching natural '
 'language processing and generative models for text, images, and audio. They '
 'are developing advanced language models like GPT-3 and image generation '
 'models like DALL-E.')
############################################################
('Condensed:  Based on the article, OpenAI is currently researching advanced '
 'language processing to generate high-quality text and images. They are '
 'developing systems like GPT-3 and DALL-E that can generate text and images '
 'from instructions to better capture user intent.')


In [124]:
from rouge_score import rouge_scorer

rs = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

rs.score(original, condensed)


{'rougeL': Score(precision=0.5365853658536586, recall=0.6285714285714286, fmeasure=0.5789473684210527)}